# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

coordinates = {
    "Lat": (city_data_df["Lat"]),
    "Lng": (city_data_df["Lng"]),
    "City": (city_data_df["City"]),
    "Humidity": (city_data_df["Humidity"])
}

coordinates_df = pd.DataFrame(coordinates)

humidity_map = coordinates_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)
humidity_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) ]

narrowed_city_df





,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
19,19,puerto ayora,-0.7393,-90.3518,22.98,82,41,1.79,EC,1666108242
20,20,avarua,-21.2078,-159.7750,22.03,73,20,2.57,CK,1666108220
27,27,bambous virieux,-20.3428,57.7575,22.14,68,75,3.60,MU,1666108246
38,38,labuhan,-6.8844,112.2051,25.24,88,100,2.74,ID,1666108254
...,...,...,...,...,...,...,...,...,...,...
563,563,ilhabela,-23.7781,-45.3581,22.67,88,95,2.32,BR,1666108558
566,566,nhulunbuy,-12.2333,136.7667,26.34,78,88,0.00,AU,1666108559
567,567,sangamner,19.5667,74.2167,23.82,85,100,2.28,IN,1666108559
569,569,ranong,9.9658,98.6348,24.34,93,100,1.44,TH,1666108330


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:


hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

hotel_df

,City,Country,Lat,Lng,Humidity
0,faya,SA,18.3851,42.4509,35
19,puerto ayora,EC,-0.7393,-90.3518,82
20,avarua,CK,-21.2078,-159.7750,73
27,bambous virieux,MU,-20.3428,57.7575,68
38,labuhan,ID,-6.8844,112.2051,88
...,...,...,...,...,...
563,ilhabela,BR,-23.7781,-45.3581,88
566,nhulunbuy,AU,-12.2333,136.7667,78
567,sangamner,IN,19.5667,74.2167,85
569,ranong,TH,9.9658,98.6348,93


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df


# Set parameters to search for a hotel
radius = 1000
params = {"categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20}
    # YOUR CODE HERE


 # Print a message to follow up the hotel search
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for i, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[i, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[i, "Hotel Name"] = "Hotel Not Found"
    # Log the search results
    print(f"{hotel_df.loc[i, 'City']} - nearest hotel: {hotel_df.loc[i, 'Hotel Name']}")
# Display sample data
hotel_df   
    
# Print a message to follow up the hotel search
print("Starting hotel search")
 

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: Hotel Not Found
puerto ayora - nearest hotel: Hotel Not Found
avarua - nearest hotel: Hotel Not Found
bambous virieux - nearest hotel: Hotel Not Found
labuhan - nearest hotel: Hotel Not Found
kapaa - nearest hotel: Hotel Not Found
hilo - nearest hotel: Hotel Not Found
businga - nearest hotel: Hotel Not Found
lasa - nearest hotel: Hotel Not Found
vaini - nearest hotel: Hotel Not Found
banda - nearest hotel: Hotel Not Found
caravelas - nearest hotel: Hotel Not Found
mahebourg - nearest hotel: Hotel Not Found
makakilo city - nearest hotel: Hotel Not Found
rafaela - nearest hotel: Hotel Not Found
saint-philippe - nearest hotel: Hotel Not Found
ponta do sol - nearest hotel: Hotel Not Found
sao filipe - nearest hotel: Hotel Not Found
muncar - nearest hotel: Hotel Not Found
bocana de paiwas - nearest hotel: Hotel Not Found
ambon - nearest hotel: Hotel Not Found
biak - nearest hotel: Hotel Not Found
kahului - nearest hotel: Hotel Not Found
innisfail 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,Hotel Not Found
19,puerto ayora,EC,-0.7393,-90.3518,82,Hotel Not Found
20,avarua,CK,-21.2078,-159.7750,73,Hotel Not Found
27,bambous virieux,MU,-20.3428,57.7575,68,Hotel Not Found
38,labuhan,ID,-6.8844,112.2051,88,Hotel Not Found
...,...,...,...,...,...,...
563,ilhabela,BR,-23.7781,-45.3581,88,Hotel Not Found
566,nhulunbuy,AU,-12.2333,136.7667,78,Hotel Not Found
567,sangamner,IN,19.5667,74.2167,85,Hotel Not Found
569,ranong,TH,9.9658,98.6348,93,Hotel Not Found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:


# Configure the map plot
m_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
m_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)